In [7]:
import glob
import pandas as pd
import numpy as np
import pickle

In [8]:
data_path = "/Users/jurajkapasny/Data/sk_text_for_api"

In [9]:
def load_data(path):
    allFiles = glob.glob(path + "/*.csv")
    print "Number of files to process: %d" %len(allFiles)
    df = pd.DataFrame()
    list_ = []
    i = 0
    for file_ in allFiles:
        if (file_.find("articles") != -1) & (file_.find("temo") == -1):
            i = i + 1
            if i%50 == 0:
                print "Processing file %d" %i
            temp = pd.read_csv(file_,sep = "|",index_col=None, header=0, parse_dates=True, low_memory=False)
            list_.append(temp)
    df = pd.concat(list_).reset_index(drop = True)
    return df

In [10]:
df = load_data(data_path)

Number of files to process: 189
Processing file 50
Processing file 100
Processing file 150


In [11]:
df.head()

,label,summary,text
0,NaN,"Placebo má 20 rokov, oslavy boli aj v Bratisla...",\nPozrite si atmosféru z víkendového koncertu ...
1,NaN,"Treba zakázať Uber len preto, že nezapadá do s...",\nV jednom svete najskôr všetko dovolí parlame...
2,techbox,Samsung už v minulosti skúšal zaujať SUHD tele...,"\nVerím, že mnohí z vás v tom majú jasno, no u..."
3,NaN,Izrael opäť zaútočil na sýrske vojenské ciele ...,"\n po tom, čo na ním kontrolované Golanské výš..."
4,NaN,Basketbalistky Španielska sa stali víťazkami m...,\n v Česku. Vo finálovom súboji triumfovali na...


In [12]:
df_clean = df[(df.label.notnull()) & (df.text.notnull())][["label","text"]]

In [13]:
df_clean.head()

,label,text
2,techbox,"\nVerím, že mnohí z vás v tom majú jasno, no u..."
5,techbox,\nGmail na prispôsobenie reklamy používa prehl...
9,techbox,"\nLen máloktorá firma zmenila svet internetu, ..."
116,techbox,\nEurópska únia v poslednom čase predstavuje j...
128,techbox,\nNemecká spoločnosť Loewe predstavila novú sé...


In [14]:
docs = df_clean.text.tolist()

### Data Processing

In [15]:
from nltk.stem.wordnet import WordNetLemmatizer
import string

In [16]:
text_file = open("/Users/jurajkapasny/nltk_data/corpora/stopwords-sk/stopwords-sk.json", "r")
lines = text_file.read()
stopwords_sk = lines.replace('"','').split(",")

In [17]:
for i in range(len(stopwords_sk)):
    stopwords_sk[i] = stopwords_sk[i].replace("”","")
    stopwords_sk[i] = stopwords_sk[i].replace("“","")

In [18]:
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stopwords_sk])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    num_free = ''.join(ch for ch in punc_free if ch not in ['0','1','2','3','4','5','6','7','8','9'])
    normalized = " ".join(lemma.lemmatize(word) for word in num_free.split())
    no_short = " ".join(word for word in normalized.split() if len(word) > 3)
    return no_short

In [19]:
doc_clean = [clean(doc).split() for doc in docs]  

In [ ]:
pr

In [35]:
for i in doc_clean[1]:
    print i

gmail
prispôsobenie
reklamy
používa
prehliadanie
správ
doručené
mailovej
schránky
konca
tohto
roka
definitívne
odzvoniť
mailový
klient
google
patrí
najpopulárnejšie
produkty
spoločnosti
niektorí
môžu
namietať
celý
vyhľadávací
gigant
veľký
obchod
reklamou
zberom
osobných
údajov
pozrite
sigmailify
funkcie
gmail
gmailcom
gmail
sebe
zakomponovaný
skener
hľadá
kľúčové
slová
základe
nich vie
veľký
brat
prispôsobiť
reklamy
každému
osobne
napriek
tomu
prehliadanie
starosti
stroj
človek
majú
niektorí
miliardy
používateľov
zásah
súkromia
ťažké
srdce
malo
chvíľu
zmeniť
spoločnosť
zrušila
aktivitu
business
verzii
gmailu
spadá
skupiny
suite
zmeny
dočkáme
bežní
používatelia
pravidlo
nadobudne
účinnosť
konca
tohto
roka
pozrite
tipov
oceníte
používate
gmail
samozrejme
neznamená
ostatné
služby
google
iných
spoločností
nebudú
dáta
zbierať
vyhľadávač
ďalej
posvieti
história
prehliadania
stránky
navštevujeme
videá
sledujeme
youtube
ďalšie
zdrojgoogle
blog


In [34]:
print doc_clean[1]


['gmail', 'prisp\xc3\xb4sobenie', 'reklamy', 'pou\xc5\xbe\xc3\xadva', 'prehliadanie', 'spr\xc3\xa1v', 'doru\xc4\x8den\xc3\xa9', 'mailovej', 'schr\xc3\xa1nky', 'konca', 'tohto', 'roka', 'definit\xc3\xadvne', 'odzvoni\xc5\xa5', 'mailov\xc3\xbd', 'klient', 'google', 'patr\xc3\xad', 'najpopul\xc3\xa1rnej\xc5\xa1ie', 'produkty', 'spolo\xc4\x8dnosti', 'niektor\xc3\xad', 'm\xc3\xb4\xc5\xbeu', 'namieta\xc5\xa5', 'cel\xc3\xbd', 'vyh\xc4\xbead\xc3\xa1vac\xc3\xad', 'gigant', 've\xc4\xbek\xc3\xbd', 'obchod', 'reklamou', 'zberom', 'osobn\xc3\xbdch', '\xc3\xbadajov', 'pozrite', 'sigmailify', 'funkcie', 'gmail', 'gmailcom', 'gmail', 'sebe', 'zakomponovan\xc3\xbd', 'skener', 'h\xc4\xbead\xc3\xa1', 'k\xc4\xbe\xc3\xba\xc4\x8dov\xc3\xa9', 'slov\xc3\xa1', 'z\xc3\xa1klade', 'nich\xc2\xa0vie', 've\xc4\xbek\xc3\xbd', 'brat', 'prisp\xc3\xb4sobi\xc5\xa5', 'reklamy', 'ka\xc5\xbed\xc3\xa9mu', 'osobne', 'napriek', 'tomu', 'prehliadanie', 'starosti', 'stroj', '\xc4\x8dlovek', 'maj\xc3\xba', 'niektor\xc3\xad', 'mil

In [44]:
# Importing Gensim
import gensim
from gensim import corpora

/Applications/anaconda/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [45]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

In [46]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [47]:
# Creating the object for LDA model using gensim library
lda = gensim.models.ldamodel.LdaModel

In [48]:
# Running and Trainign LDA model on the document term matrix.
ldamodel = lda(doc_term_matrix, num_topics=20, id2word = dictionary, passes=50)

In [49]:
pickle.dump( ldamodel, open( "/Users/jurajkapasny/Code/GitHub/jurajkapasny/text_analytics/LDA.p", "wb" ) )

In [2]:
ldamodel = pickle.load( open( "/Users/jurajkapasny/Code/GitHub/jurajkapasny/text_analytics/LDA.p", "rb" ) )

/Applications/anaconda/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [36]:
print ldamodel.print_topics(num_topics=20, num_words=10)[0][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[1][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[2][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[3][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[4][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[5][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[6][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[7][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[8][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[9][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[10][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[11][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[12][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[13][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[14][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[15][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[16][1]
print ""
print ldamodel.print_topics(num_topics=20, num_words=10)[17][1]
print "" 
print ldamodel.print_topics(num_topics=20, num_words=10)[18][1]
print "" 
print ldamodel.print_topics(num_topics=20, num_words=10)[19][1]
print "" 

0.005*"zadné" + 0.005*" km" + 0.004*"príspevok" + 0.004*"zdieľa" + 0.004*"sagan" + 0.004*"etapa" + 0.003*"júl" + 0.003*"              " + 0.002*"august" + 0.002*"september"

0.006*"krajiny" + 0.005*"európskej" + 0.004*"povedal" + 0.004*"slovensko" + 0.004*"utečencov" + 0.004*"únie" + 0.004*"krajín" + 0.003*"proti" + 0.003*"európy" + 0.002*"teraz"

0.005*"roku" + 0.004*"spoločnosti" + 0.004*"spoločnosť" + 0.004*" miliónov" + 0.003*"v roku" + 0.003*"firmy" + 0.003*"banky" + 0.003*"ceny" + 0.003*"roka" + 0.003*"firma"

0.011*"strany" + 0.007*"vlády" + 0.006*"strana" + 0.005*"povedal" + 0.005*"parlamentu" + 0.005*"voľbách" + 0.005*"fico" + 0.004*"predseda" + 0.004*"voľby" + 0.004*"volieb"

0.003*"počas" + 0.003*"môžu" + 0.003*"telo" + 0.002*"vody" + 0.002*"majú" + 0.002*"najmä" + 0.002*"ochorenia" + 0.002*"proti" + 0.002*"treba" + 0.002*"napríklad"

0.007*"mäso" + 0.005*"pridáme" + 0.004*" sj" + 0.004*"soľ" + 0.003*"mäsa" + 0.003*"postup" + 0.003*"necháme" + 0.003*" minút" + 0.003*"korenie

In [51]:
print ldamodel.print_topics(num_topics=20, num_words=10)[19]
print "" 

(19, u'0.008*"valec" + 0.008*"kolies" + 0.006*"zadn\xfdch" + 0.005*"renault" + 0.004*"kombi" + 0.003*"koles\xe1" + 0.003*"golf" + 0.002*"nissan" + 0.002*"pneumatiky" + 0.002*"suzuki"')

